In [72]:
import pandas as pd
import numpy as np

In [73]:
df = pd.read_csv('data-files/movies/metadata_clean.csv')
df.head()

,title,genres,runtime,vote_average,vote_count,year
0,Toy Story,"['animation', 'comedy', 'family']",81.0,7.7,5415.0,1995
1,Jumanji,"['adventure', 'fantasy', 'family']",104.0,6.9,2413.0,1995
2,Grumpier Old Men,"['romance', 'comedy']",101.0,6.5,92.0,1995
3,Waiting to Exhale,"['comedy', 'drama', 'romance']",127.0,6.1,34.0,1995
4,Father of the Bride Part II,['comedy'],106.0,5.7,173.0,1995


In [74]:
orig_df = pd.read_csv('data-files/movies/movies_metadata.csv')
orig_df['overview']

C:\ProgramData\Miniconda3\envs\hoseo\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


0        Led by Woody, Andy's toys live happily in his ...
1        When siblings Judy and Peter discover an encha...
2        A family wedding reignites the ancient feud be...
3        Cheated on, mistreated and stepped on, the wom...
4        Just when George Banks has recovered from his ...
                               ...                        
45461          Rising and falling between a man and woman.
45462    An artist struggles to finish his work while a...
45463    When one of her hits goes wrong, a professiona...
45464    In a small town live two brothers, one a minis...
45465    50 years after decriminalisation of homosexual...
Name: overview, Length: 45466, dtype: object

In [75]:
df['overview'] = orig_df['overview']
df['id'] = orig_df['id']
df['overview'] = df['overview'].fillna('') # na값 대체

In [76]:
df.head() # 영화의 텍스트 정보를 통해 유사성 찾아보자! 

,title,genres,runtime,vote_average,vote_count,year,overview,id
0,Toy Story,"['animation', 'comedy', 'family']",81.0,7.7,5415.0,1995,"Led by Woody, Andy's toys live happily in his ...",862
1,Jumanji,"['adventure', 'fantasy', 'family']",104.0,6.9,2413.0,1995,When siblings Judy and Peter discover an encha...,8844
2,Grumpier Old Men,"['romance', 'comedy']",101.0,6.5,92.0,1995,A family wedding reignites the ancient feud be...,15602
3,Waiting to Exhale,"['comedy', 'drama', 'romance']",127.0,6.1,34.0,1995,"Cheated on, mistreated and stepped on, the wom...",31357
4,Father of the Bride Part II,['comedy'],106.0,5.7,173.0,1995,Just when George Banks has recovered from his ...,11862


In [77]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [78]:
tfidf = TfidfVectorizer(stop_words='english') # stop_words = 관사(a, the), he, she 등 의미 부여하기 어려운 것은 제거하고 나머지로 계산하겠다
tfidf_matrix = tfidf.fit_transform(df['overview']) # 전체 문장 45466중 75827개의 단어가 구성되어 있음

In [79]:
tfidf_matrix2 = tfidf_matrix.astype('float32')
print(tfidf_matrix.dtype, tfidf_matrix2.dtype)

float64 float32


In [80]:
from sklearn.metrics.pairwise import linear_kernel # 코사인 유사도 계산

In [81]:
cosine_sim = linear_kernel(tfidf_matrix2,tfidf_matrix2) # 왼쪽과 오른쪽의 유사도 비교

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\ProgramData\Miniconda3\envs\hoseo\lib\site-packages\IPython\core\interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-81-800045df93bb>", line 1, in <module>
    cosine_sim = linear_kernel(tfidf_matrix2,tfidf_matrix2) # 왼쪽과 오른쪽의 유사도 비교
  File "C:\ProgramData\Miniconda3\envs\hoseo\lib\site-packages\sklearn\metrics\pairwise.py", line 1003, in linear_kernel
    return safe_sparse_dot(X, Y.T, dense_output=dense_output)
  File "C:\ProgramData\Miniconda3\envs\hoseo\lib\site-packages\sklearn\utils\validation.py", line 73, in inner_f
    return f(**kwargs)
  File "C:\ProgramData\Miniconda3\envs\hoseo\lib\site-packages\sklearn\utils\extmath.py", line 153, in safe_sparse_dot
    ret = a @ b
  File "C:\ProgramData\Miniconda3\envs\hoseo\lib\site-packages\scipy\sparse\base.py", line 560, in __matmul__
    return self.__mul__(other)
  File "C:\ProgramData\Miniconda3\envs\hoseo\lib\site-p

KeyboardInterrupt: 

In [ ]:
consine_sim # 본인은 1, 나머지 대칭 
# cosine_sim.shape # 유사도를 matix로 정의 

In [ ]:
indices = pd.Series(df.index, index=df['title']).drop_duplicates() # drop_duplicates() -> 중복되는 데이터 제거 # title index 순서대로 값으로 처리
indices.head() # toy story는 0번, Jumanji는 1번 ...

In [ ]:
idx = indices['Father of the Bride Part II']
print(idx) # 이 영화는 4번 째에 있다

In [ ]:
# cosine_sim.shape
sim_scores = list(enumerate(cosine_sim[idx])) # 0번은 0, 2번 영화와 similarity 0.025

sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse=True) # x[1] : 2열 # reverse=True : 내림차순

In [ ]:
top_sim_scores = sim_scores[1:11] # 본인 것이 가장 높음

In [ ]:
top_sim_scores

In [ ]:
movie_indices = [ sim_score[0] for sim_score in top_sim_scores ]
movie_indices

In [ ]:
df['title'].iloc[movie_indices]

In [ ]:
#####

In [93]:
credits_df = pd.read_csv('data-files/movies/credits.csv')
keywords_df = pd.read_csv('data-files/movies/keywords.csv')

In [84]:
credit_df.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [85]:
keywords_df.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [94]:
def clean_id(x):
    try:
        return int(x)
    except:
        return np.nan

In [95]:
df['id'].dtype #문자열

dtype('int32')

In [104]:
df['id'] = df['id'].apply(clean_id)
df = df[df['id'].notnull()]

In [105]:
df['id'] = df['id'].astype('int')
credits_df['id'] = credits_df['id'].astype('int')
keywords_df['id'] = keywords_df['id'].astype('int')

In [106]:
df = df.merge(credits_df, on='id').merge(keywords_df, on='id')

In [107]:
df.head()

,title,genres,runtime,vote_average,vote_count,year,overview,id,cast_x,crew_x,keywords_x,cast_y,crew_y,keywords_y,cast,crew,keywords
0,Toy Story,"['animation', 'comedy', 'family']",81.0,7.7,5415.0,1995,"Led by Woody, Andy's toys live happily in his ...",862,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,Jumanji,"['adventure', 'fantasy', 'family']",104.0,6.9,2413.0,1995,When siblings Judy and Peter discover an encha...,8844,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1...","[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1...","[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,Grumpier Old Men,"['romance', 'comedy']",101.0,6.5,92.0,1995,A family wedding reignites the ancient feud be...,15602,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392...","[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392...","[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,Waiting to Exhale,"['comedy', 'drama', 'romance']",127.0,6.1,34.0,1995,"Cheated on, mistreated and stepped on, the wom...",31357,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':...","[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':...","[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,Father of the Bride Part II,['comedy'],106.0,5.7,173.0,1995,Just when George Banks has recovered from his ...,11862,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...","[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...","[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [108]:
# Convert the stringified objects into the native python objects
from ast import literal_eval

features = ['cast', 'crew', 'keywords', 'genres'] # 문자열로 여러개 들어간 것을 푼다
for feature in features:
    df[feature] = df[feature].apply(literal_eval)

In [109]:
#Print the first cast member of the first movie in df
df.iloc[0]['crew'][0]

{'credit_id': '52fe4284c3a36847f8024f49',
 'department': 'Directing',
 'gender': 2,
 'id': 7879,
 'job': 'Director',
 'name': 'John Lasseter',
 'profile_path': '/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg'}

In [111]:
# Extract the director's name. If director is not listed, return NaN # 감독만 뽑아내기
def get_director(x):
    for crew_member in x:
        if crew_member['job'] == 'Director':
            return crew_member['name']
    return np.nan # job이 Director인 것이 아니면 nan으로


In [113]:
#Define the new director feature
df['director'] = df['crew'].apply(get_director)

#Print the directors of the first five movies
df['director'].head(10)


0      John Lasseter
1       Joe Johnston
2      Howard Deutch
3    Forest Whitaker
4      Charles Shyer
5       Michael Mann
6     Sydney Pollack
7       Peter Hewitt
8        Peter Hyams
9    Martin Campbell
Name: director, dtype: object

In [114]:
# Returns the list top 3 elements or entire list; whichever is more.
def generate_list(x):
    if isinstance(x, list): # x가 list라면 밑의 결과를
        names = [i['name'] for i in x] # name만 뽑아내겠다
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names

    #Return empty list in case of missing/malformed data # list가 아니라면 [] 빈 리스트로 뽑아내겠다
    return []

In [115]:
#Apply the generate_list function to cast and keywords
df['cast'] = df['cast'].apply(generate_list)
df['keywords'] = df['keywords'].apply(generate_list)

In [117]:
df['cast'][0], df['keywords'][0]

(['Tom Hanks', 'Tim Allen', 'Don Rickles'], ['jealousy', 'toy', 'boy'])

In [118]:
#Only consider a maximum of 3 genres
df['genres'] = df['genres'].apply(lambda x: x[:3]) # na 데이터가 많아서 많이 나오는 3 장르만

In [119]:
# Print the new features of the first 5 movies along with title
df[['title', 'cast', 'director', 'keywords', 'genres']].head()

,title,cast,director,keywords,genres
0,Toy Story,"[Tom Hanks, Tim Allen, Don Rickles]",John Lasseter,"[jealousy, toy, boy]","[animation, comedy, family]"
1,Jumanji,"[Robin Williams, Jonathan Hyde, Kirsten Dunst]",Joe Johnston,"[board game, disappearance, based on children'...","[adventure, fantasy, family]"
2,Grumpier Old Men,"[Walter Matthau, Jack Lemmon, Ann-Margret]",Howard Deutch,"[fishing, best friend, duringcreditsstinger]","[romance, comedy]"
3,Waiting to Exhale,"[Whitney Houston, Angela Bassett, Loretta Devine]",Forest Whitaker,"[based on novel, interracial relationship, sin...","[comedy, drama, romance]"
4,Father of the Bride Part II,"[Steve Martin, Diane Keaton, Martin Short]",Charles Shyer,"[baby, midlife crisis, confidence]",[comedy]


In [120]:
# Function to sanitize data to prevent ambiguity. It removes spaces and converts to lowercase
def sanitize(x):
    if isinstance(x, list):
        #Strip spaces and convert to lowercase
        return [str.lower(i.replace(" ", "")) for i in x] # 띄어쓰기 공백이 있다면 없애라
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [121]:
#Apply the generate_list function to cast, keywords, director and genres
for feature in ['cast', 'director', 'genres', 'keywords']:
    df[feature] = df[feature].apply(sanitize)

In [123]:
df.head(1)

,title,genres,runtime,vote_average,vote_count,year,overview,id,cast_x,crew_x,keywords_x,cast_y,crew_y,keywords_y,cast,crew,keywords,director,soup
0,Toy Story,"[animation, comedy, family]",81.0,7.7,5415.0,1995,"Led by Woody, Andy's toys live happily in his ...",862,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...","[tomhanks, timallen, donrickles]","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[jealousy, toy, boy]",johnlasseter,jealousy toy boy tomhanks timallen donrickles ...


In [124]:
#Function that creates a soup out of the desired metadata
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres']) # 공백으로 띄어서 keywords 묶고 공백으로 띄어서 cast 공백으로 띄어서 director

In [125]:
# Create the new soup feature
df['soup'] = df.apply(create_soup, axis=1)

In [126]:
df['soup'][:2]

0    jealousy toy boy tomhanks timallen donrickles ...
1    boardgame disappearance basedonchildren'sbook ...
Name: soup, dtype: object

In [128]:
df.to_csv('data-files/soup.csv', index=False)

In [130]:
#######################

In [1]:
import pandas as pd

In [2]:
df2 = pd.read_csv('data-files/soup.csv')

In [3]:
df2['soup'][0]

'jealousy toy boy tomhanks timallen donrickles johnlasseter animation comedy family'